In [ ]:
#!pip install hyperopt

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [ ]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [ ]:
ls

data/  LICENSE  logs/  README.md


In [ ]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [ ]:
if y_train.ndim == 1 : y_train = to_categorical(y_train)
if y_test.ndim == 1 : y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [ ]:
def train_model(model,X_train, y_train, params_fit={} ):
  model.compile(loss = 'categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])
  #sciezka do katalogu w którym będą trzymane logi
  logdir= os.path.join("logs",datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs= params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data= params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]
      )
  return model
  

def predict(model_trained, X_test, y_test, scoring= accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [ ]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(), 

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')

])
  
#train_and_predict ( get_cnn_v5(input_shape, num_classes) )

In [ ]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train, params_fit={})
predict(model_trained, X_test, y_test)


Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.8367 - accuracy: 0.2474 - val_loss: 1.2990 - val_accuracy: 0.5896
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.9227 - accuracy: 0.7150 - val_loss: 0.1985 - val_accuracy: 0.9394
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.3761 - accuracy: 0.8865 - val_loss: 0.0651 - val_accuracy: 0.9807
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2259 - accuracy: 0.9326 - val_loss: 0.0442 - val_accuracy: 0.9882
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1707 - accuracy: 0.9498 - val_loss: 0.0379 - val_accuracy: 0.9897


0.9609977324263038

In [ ]:
  def get_model(params):
    return Sequential([
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_one']),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_two']),

      Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_three']),

      Flatten(), 

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_one']),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_two']),

      Dense(num_classes, activation='softmax')

  ])
  

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [ ]:
model_trained.evaluate(X_test,y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.2140 - accuracy: 0.9610


0.9609977602958679

In [ ]:
def func_obj(params): 
  model= get_model(params)
  model.compile(loss = 'categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs= 5,
      verbose =0  
      )
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy{}'.format(accuracy))

  #funkcja bedzie minimalizowac, mnozymy przez -1 żeby dążyło do -1.0 - czyli do 100%
  return {'loss' : -accuracy, 'status' : STATUS_OK, 'model' : model}

In [ ]:
space = {
    'batch_size':hp.quniform('batch_size',100,200,10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one',0.3,0.6),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two',0.3,0.6),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three',0.3,0.6),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one',0.3,0.6),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two',0.3,0.6)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials()
)

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.32908594438108296, 'dropout_cnn_block_three': 0.41200139472711816, 'dropout_cnn_block_two': 0.3542325773291066, 'dropout_dense_block_one': 0.33016550633968145, 'dropout_dense_block_two': 0.5001640364626654}
accuracy0.9482993483543396
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.5603211061090587, 'dropout_cnn_block_three': 0.4451335209961583, 'dropout_cnn_block_two': 0.31354896843503993, 'dropout_dense_block_one': 0.3685286605555014, 'dropout_dense_block_two': 0.5369386068124538}
accuracy0.8553287982940674
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4031748783158313, 'dropout_cnn_block_three': 0.4136042751838985, 'dropout_cnn_block_two': 0.33629908943422604, 'dropout_dense_block_one': 0.3147014778821827, 'dropout_dense_block_two': 0.40433470340495703}
accuracy0.9648526310920715
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.5102374868876184, 'dropout_cnn_block_three': 0.4534898886319348, 'dropout_cnn_block_two': 0.4988644487579